# **IP1_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better.

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions.

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i.  """


In [6]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [7]:
print(response1.choices[0].message.content)

Variables:

1. Xi,j: Number of proton fractions that patient i receives. Here, i ranges from 1 to 17 (since there are 17 patients) and j ranges from 0 to 15 (since the total number of fractions for each patient is 15 and any number from 0 to 15 of these fractions can be proton fractions).

2. BEDi,j: The Biological Equivalent Dose score when patient i receives j proton fractions and 15-j photon fractions. Here, i ranges from 1 to 17 and j ranges from 0 to 15.

3. C: The total capacity of proton fractions which is given as 100. 

To note: All Xi,j are integer variables since the number of proton fractions given to a patient must be a whole number.


In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [9]:
Markdown(response2.choices[0].message.content)

The objective function for this optimization problem is to maximize the total Biological Equivalent Dose (BED) scores for all the patients. Thus the objective function is:

Maximize ∑∑BEDi,j * Xi,j 
for i = 1 to 17 and j = 0 to 15.

In [10]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [11]:
Markdown(response3.choices[0].message.content)

Constraints:

1. The Total Proton Fractions Constraint: 
The sum of all proton fractions across all patients should not exceed the total capacity C.
∑∑Xi,j <= C, 
for i = 1 to 17 and j = 0 to 15.

2. The Patient Fraction Limit Constraint: 
Each patient i receives a total of 15 fractions which can be either proton or photon. Therefore, the sum of all types of fractions for each patient i must equal 15.
∑Xi,j = 15, for each i where i ranges from 1 to 17.

3. Non-negativity Constraint: 
Since we can't have a negative number of treatments, all Xi,j are greater than or equal to 0.
Xi,j >= 0, for all i, j.
 
4. Integer Constraint: 
As the fractions for each patient must be a whole number, Xi,j must be integers.
Xi,j ∈ Z+, for all i, j. Where Z+ represents the set of all positive integers including zero.

### **Generate Pyomo Code**

In [12]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [13]:
Markdown(response4.choices[0].message.content)

First, we need to install the pyomo and glpk libraries using pip. Glpk is the solver that pyomo uses for integer programming.

```python
!pip install pyomo
!apt-get install -y -qq glpk-utils
```

Next, we define the pyomo model .

```python
from pyomo.environ import *

#Lists of the patients and the possible amount of fraction
Patients = [i for i in range(1, 18)]
Fractions = [j for j in range(0, 16)]

#Create a sample data for BED score
import random
BED = {(i,j):random.randint(50, 100) for i in Patients for j in Fractions}

#Initialize model
model = ConcreteModel()

#Define Variables
model.X = Var(Patients, Fractions, within=NonNegativeIntegers)

#Objective Function
model.obj = Objective(expr=sum(BED[i,j]*model.X[i,j] for i in Patients for j in Fractions), sense=maximize)

#Constraints
model.total_proton_fractions = Constraint(expr=sum(model.X[i,j] for i in Patients for j in Fractions) <= 100)

def patient_fraction_limit(m, i):
    return sum(m.X[i,j] for j in Fractions) == 15
model.patient_fraction_limit_con = Constraint(Patients, rule=patient_fraction_limit)

#Solver
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()
```

In the sample data, we randomly assigned BED scores to each patient and fraction pair ranging from 50 to 100. The objective function maximizes the total BED score. Constraints ensure that the total number of proton fractions doesn't exceed the clinic's capacity and each patient receives exactly 15 fractions. The model assumes that each patient must receive treatment, hence all patients will be given between 0 and 15 treatments. The model doesn't dictate whether a patient should have proton or photon treatment, but using the BED score ratio, the model tries to allocate as much proton treatment as it can until the clinic's proton treatment total capacity is reached.

### **Run the code Generated by GPT4**

In [17]:
from pyomo.environ import *

# Lists of the patients and the possible amount of fraction
Patients = [i for i in range(1, 18)]
Fractions = [j for j in range(0, 16)]

# Create a sample data for BED score
import random
BED = {(i,j):random.randint(50, 100) for i in Patients for j in Fractions}

# Initialize model
model = ConcreteModel()

# Define Variables
model.X = Var(Patients, Fractions, within=NonNegativeIntegers)

# Objective Function
model.obj = Objective(expr=sum(BED[i,j]*model.X[i,j] for i in Patients for j in Fractions), sense=maximize)

# Constraints
model.total_proton_fractions = Constraint(expr=sum(model.X[i,j] for i in Patients for j in Fractions) <= 100)

def patient_fraction_limit(m, i):
    return sum(m.X[i,j] for j in Fractions) == 15
model.patient_fraction_limit_con = Constraint(Patients, rule=patient_fraction_limit)

# Solver
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 272
  Number of nonzeros: 544
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.005800485610961914


In [18]:
from pyomo.environ import *
import pandas as pd

# Lists of the patients and the possible amount of fraction
Patients = [i for i in range(1, 18)]
Fractions = [j for j in range(0, 16)]

# Create a sample data for BED score
df = pd.read_csv('ProblemData.csv', header=None)
BED_data= {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED_data[(i+1, j)] = dose
#End

# Initialize model
model = ConcreteModel()

# Define Variables
model.X = Var(Patients, Fractions, within=NonNegativeIntegers)

# Objective Function
model.obj = Objective(expr=sum(BED[i,j]*model.X[i,j] for i in Patients for j in Fractions), sense=maximize)

# Constraints
model.total_proton_fractions = Constraint(expr=sum(model.X[i,j] for i in Patients for j in Fractions) <= 100)

def patient_fraction_limit(m, i):
    return sum(model.X[i,j] for j in Fractions) == 15
model.patient_fraction_limit_con = Constraint(Patients, rule=patient_fraction_limit)

# Solver
SolverFactory('glpk', executable='/usr/bin/glpsol').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 18
  Number of variables: 272
  Number of nonzeros: 544
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.005970954895019531


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**